In [3]:
# basic imports
import numpy as np
import pandas as pd

# demos for visualization
from nilearn import datasets
from nilearn import plotting
from nilearn.connectome import GroupSparseCovariance
from nilearn.connectome import ConnectivityMeasure

# for ML
from sklearn.metrics import mutual_info_score

# os library
from os.path import exists, join

C:\Users\simon\Anaconda3\lib\site-packages\nilearn\datasets\__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [4]:
data_path = join(".", "data") if exists(join(".", "data")) \
    else join("..", "input", "statistical-learning-sapienza-spring-2021")

In [27]:
train = pd.read_csv(join(data_path, 'train.csv'))
test = pd.read_csv(join(data_path, 'test.csv'))

In [13]:
len(train.id.values)
len(train.index.values)

600

In [14]:
def get_data(entry):
    id = entry.id
    variables = entry[['var1', 'var2', 'var3']].to_numpy()
    timeseries = entry[5 if "y" in entry.index else 4:].to_numpy(dtype=float).reshape((115, 116), order="F")
    if "y" in entry.index: return id, entry.y, variables, timeseries
    else: return id, variables, timeseries

In [15]:
time_series = train.apply(lambda row: get_data(row)[-1], axis=1).to_list()

GroupSparseCovariance()

GroupSparseCovariance()

In [28]:
time_series_test = test.apply(lambda row: get_data(row)[-1], axis=1).to_list()

In [ ]:
def apply_connectivity(time_series, vect=True, diag=True):

    correlation_measure = ConnectivityMeasure(kind='partial correlation',
                                              vectorize = vect,
                                              discard_diagonal=diag)

    return correlation_measure.fit_transform(time_series)

In [30]:
correlation_matrices = apply_connectivity(time_series)
correlation_test = apply_connectivity(time_series_test)

In [18]:
correlation_matrices


array([[ 0.00637566,  0.01222776, -0.0257102 , ..., -0.02211248,
         0.05410901,  0.1392692 ],
       [ 0.0222867 ,  0.01343907, -0.02063951, ...,  0.07509686,
        -0.02293725, -0.07659659],
       [ 0.01665948,  0.00562362,  0.00429343, ..., -0.0018096 ,
         0.15409605,  0.04340404],
       ...,
       [ 0.03150218,  0.04496332,  0.02761484, ...,  0.20413376,
         0.14613533,  0.1512237 ],
       [ 0.04014914,  0.03154995, -0.04058971, ...,  0.01371451,
         0.06975417, -0.23536263],
       [ 0.01643078, -0.01265283, -0.00270768, ..., -0.01242908,
         0.06171285,  0.18588571]])

In [32]:
pd.concat([train.iloc[:, :5], pd.DataFrame(correlation_matrices)], axis=1).to_csv('train_corr.csv')
pd.concat([test.iloc[:, :5], pd.DataFrame(correlation_test)], axis=1).to_csv('test_corr.csv')